In [1]:
import numpy as np
import cs_vqe as c
from copy import deepcopy as copy

import ast
import matplotlib.pyplot as plt
import os

import pickle
import datetime

In [2]:
working_dir = os.getcwd()
data_dir = os.path.join(working_dir, 'Pickle_out')

data_H_dir = os.path.join(working_dir, 'data')
data_hamiltonians_file = os.path.join(data_H_dir, 'hamiltonians.txt')

In [3]:
file_name = 'EXP_ordering_anticommuting_vs_standard_conH_2021Mar30-110555019968.pickle'
file_path = os.path.join(data_dir, file_name)
with open(file_path, 'rb') as infile:
    unitary_paritioning_of_Con_H = pickle.load(infile)
    

file_name2 = 'EXP_ordering_updated_hamiltonians_2021Mar30-110555019968.pickle'
file_path2 = os.path.join(data_dir, file_name2)
with open(file_path2, 'rb') as infile:
    updated_hamiltonians = pickle.load(infile)

In [4]:
data_csvqe_results_file = os.path.join(data_H_dir, 'csvqe_results.txt')
with open(data_csvqe_results_file, 'r') as input_file:
    csvqe_results = ast.literal_eval(input_file.read())

In [13]:
import Unitary_partitioning_LCU_method as LCU_UP
from copy import deepcopy as copy

In [6]:
# LCU check
E_LCU_dict ={}
for mol_name in unitary_paritioning_of_Con_H:
    
    N_Qubits= updated_hamiltonians[mol_name]['n_qubits']
    
    if N_Qubits>10:
        continue

    true_gs_energy = csvqe_results[mol_name][0]
    
    E_list=[]
    error_list=[]
    for n_qubits, tuple_fullH_ACset in enumerate(unitary_paritioning_of_Con_H[mol_name]):

        AC_set = tuple_fullH_ACset[1]
        if n_qubits == 0:
            Energy_I = list(AC_set[0][0].terms.values())[0] # <- no qubits!
            E_list.append(Energy_I) 
            error_list.append(abs(true_gs_energy-Energy_I))
        else:

            N_dict = {key:0 for key in AC_set}
            E_LCU = LCU_UP.LCU_linalg_Energy(AC_set,
                                      N_dict,
                                      n_qubits,
                                      atol=1e-8,
                                      rtol=1e-05,
                                      check_reduction=True) ### <--- change for paper!
            E_list.append(copy(E_LCU))
            error_list.append(abs(true_gs_energy-E_LCU))
            del E_LCU
    E_LCU_dict[mol_name] = {'Energy_list': E_list,
                            'Error_list': error_list}

In [ ]:
AC_set

In [7]:
for mol_name in E_LCU_dict:
    print(E_LCU_dict[mol_name]['Error_list'])
    print('')

[0.012805571551730122, 0.0003080609044734395, 2.220446049250313e-15, 3.552713678800501e-15]

[0.05177463186564246, 0.030864204200282686, 0.014216038765219707, 0.00032650598432049094, 0.0001759789016606561, 7.030820370346191e-11]

[0.024765804413788395, 0.01794527467613216, 0.017943621134969723, 0.012216134976930304, 0.007884547381673768, 4.440892098500626e-16]

[0.024692363259258387, 0.019275277658154844, 0.019273258881908895, 0.013363995296319153, 0.00809672421514751, 6.439293542825908e-15]

[0.023508175699287648, 0.01882399499271248, 0.01549232114093435, 0.014469429665340705, 0.002205439536466791, 0.00043708987844315317, 2.6645352591003757e-15]

[0.028901424436270418, 0.02890142443629884, 0.00705653630426184, 0.004891451939514013, 0.0008274315610492522, 0.0004116191125831392, 2.0340464061519015e-05, 1.4287944395618979e-05, 1.4210854715202004e-14]

[0.019144837831428063, 0.01914483783142984, 0.017308996341892424, 0.0036894902683179964, 0.0013358883065048843, 0.0007708111407414009, 0.0

In [46]:
### checking if R_LCU is unitary!

# import Unitary_partitioning_LCU_method as LCU_UP
# from openfermion.utils import hermitian_conjugated
# from openfermion.ops import QubitOperator

# mol_name = 'H2-O1_STO-3G_singlet'
# # mol_name = 'H3-N1_STO-3G_singlet'
# longest_key, largest_AC_set = max(unitary_paritioning_of_Con_H[mol_name][-1][1].items(), key = lambda x: len(x[1]))
# N_Qubits = updated_hamiltonians[mol_name]['n_qubits']
# N_index=0
# check_reduction=True

# R_LCU_list, Pn, gamma_l = LCU_UP.Get_R_op_list(largest_AC_set,
#                                                N_index,
#                                                N_Qubits,
#                                                check_reduction=check_reduction,
#                                                atol=1e-8, 
#                                                rtol=1e-05)


# R_LCU = QubitOperator()
# for op in R_LCU_list:
#     R_LCU+=op
# # R_LCU.renormalize()

# unitary_check = R_LCU*hermitian_conjugated(R_LCU)
# print(unitary_check)
# if unitary_check != QubitOperator('', 1):
#     raise ValueError('R_LCU is not unitary!')
# R_LCU = list(R_LCU)